In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
ds.dataset = ds.dataset.shuffle()
ds = ds.select(list(range(100)))
ds.save_images("../data/images100")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-18 17:21:03.747 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 17:21:03.755 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images100


Saving images (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 100
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,../data/images100/281365.jpg,a mens bathroom with a bunch of urinals and la...
1,../data/images100/477389.jpg,The baby elephant is walking with a small obje...
2,../data/images100/527510.jpg,A picture of a plane in a runway.
3,../data/images100/322937.jpg,Man playing tennis about to deliver a strong h...
4,../data/images100/31817.jpg,A man on a motor scooter waiting in a group of...
...,...,...
95,../data/images100/62819.jpg,A person is riding waves on a canal.
96,../data/images100/65357.jpg,"A small, black train is on its tracks by a fence."
97,../data/images100/461805.jpg,A green motorcycle parked in a parking space.
98,../data/images100/92167.jpg,A single large bird glides with wings extended.


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

PostgreSQLDatabase.create_database("my_database")

2024-12-18 17:21:04.220 | INFO     | pgsql_search.database:create_database:300 - Database 'my_database' already exists


In [4]:
with PostgreSQLDatabase("my_database") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 17:21:04.260 | INFO     | pgsql_search.database:connect:86 - Connected to database
2024-12-18 17:21:04.263 | INFO     | pgsql_search.database:setup_pgvector_extension:103 - pgvector extension initialized
2024-12-18 17:21:04.267 | INFO     | pgsql_search.database:initialize_table:125 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 17:21:04.269 | INFO     | pgsql_search.database:add_columns:174 - Added 1 new columns to image_metadata
2024-12-18 17:21:04.270 | INFO     | pgsql_search.database:add_columns:174 - Added 1 new columns to image_metadata
2024-12-18 17:21:04.277 | INFO     | pgsql_search.database:insert_dataframe:226 - Inserted batch 1 (100/100 rows)
2024-12-18 17:21:04.277 | INFO     | pgsql_search.database:insert_dataframe:230 - Successfully inserted 100 rows into image_metadata
2024-12-18 17:21:04.277 | INFO     | pgsql_search.database:disconnect:95 - Disconnected from database
